In [1]:
import torch

In [11]:
from torchvision import models, transforms
from PIL import Image

In [4]:
an = models.AlexNet()

In [7]:
rn = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /Users/alxhill/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [10]:
preprocess = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [17]:
img = Image.open("../data/raw/living_room/living.jpeg")

In [19]:
img_proc = preprocess(img)

In [20]:
img_proc

tensor([[[ 0.7762,  0.7762,  0.7762,  ...,  0.9988,  1.1187,  1.3070],
         [ 0.8104,  0.8104,  0.8104,  ...,  1.2385,  1.3755,  1.4098],
         [ 0.8961,  0.8789,  0.8789,  ...,  1.4098,  1.4098,  1.4098],
         ...,
         [-1.5185, -1.5185, -1.5185,  ...,  0.0912,  0.3823,  0.1597],
         [-1.5870, -1.5870, -1.5870,  ..., -0.3027,  0.1083,  0.3138],
         [-1.5699, -1.5528, -1.5699,  ..., -0.4739, -0.3198,  0.0741]],

        [[ 0.6254,  0.6429,  0.6254,  ...,  0.7304,  0.8529,  1.0455],
         [ 0.6779,  0.6779,  0.6779,  ...,  0.9405,  1.0805,  1.0980],
         [ 0.7304,  0.7129,  0.7304,  ...,  1.1155,  1.0980,  1.0805],
         ...,
         [-1.6155, -1.5980, -1.5980,  ..., -0.6352, -0.4426, -0.6527],
         [-1.6856, -1.6681, -1.6856,  ..., -0.9853, -0.6176, -0.4776],
         [-1.6681, -1.6681, -1.6856,  ..., -1.1604, -1.0028, -0.6527]],

        [[ 0.4439,  0.4439,  0.4265,  ...,  0.5659,  0.6531,  0.8274],
         [ 0.4788,  0.4614,  0.4614,  ...,  0

In [21]:
input_t = torch.unsqueeze(img_proc, 0)

In [24]:
rn.eval()
out_t = rn(input_t)

In [26]:
with open("../data/models/imagenet_classes.txt") as f:
    labels = [line.strip() for line in f.readlines()]

In [27]:
_, idx = torch.max(out_t, 1)

In [28]:
pct = torch.nn.functional.softmax(out_t, dim=1)[0] * 100

In [29]:
labels[idx[0]], pct[idx[0]].item()

('grand piano, grand', 40.70096969604492)

In [30]:
_, sort_idx = torch.sort(out_t, descending=True)

In [31]:
[(labels[i], pct[i].item()) for i in sort_idx[0][:5]]

[('grand piano, grand', 40.70096969604492),
 ('window shade', 15.908583641052246),
 ('home theater, home theatre', 4.891425132751465),
 ('sliding door', 4.216376304626465),
 ('entertainment center', 1.4299015998840332)]

In [32]:
midas = torch.hub.load("isl-org/MiDaS", "DPT_Hybrid")

Using cache found in /Users/alxhill/.cache/torch/hub/isl-org_MiDaS_master
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_hybrid_384.pt" to /Users/alxhill/.cache/torch/hub/checkpoints/dpt_hybrid_384.pt


  0%|          | 0.00/470M [00:00<?, ?B/s]

In [35]:
midas_transforms = torch.hub.load("isl-org/MiDaS", "transforms")

Using cache found in /Users/alxhill/.cache/torch/hub/isl-org_MiDaS_master


In [ ]:
midas.to("cpu")
midas.eval()

In [38]:
import cv2

In [41]:
img_cv = cv2.imread("../data/raw/living_room/living.jpeg")

In [42]:
with torch.no_grad():
    in_batch = midas_transforms.dpt_transform(img_cv).to("cpu")
    prediction = 

tensor([[[[-2.2319e-01, -2.2239e-01, -1.9654e-01,  ...,  1.4698e-01,
            1.3878e-01,  1.8534e-01],
          [-1.8872e-01, -2.1714e-01, -1.9653e-01,  ...,  1.4369e-01,
            1.6922e-01,  1.5970e-01],
          [-2.2690e-01, -2.3234e-01, -2.0527e-01,  ...,  1.3164e-01,
            1.4745e-01,  1.3676e-01],
          ...,
          [-7.9428e-01, -7.9817e-01, -8.1925e-01,  ..., -8.1719e-01,
           -8.1464e-01, -7.6400e-01],
          [-8.5181e-01, -7.7221e-01, -7.9221e-01,  ..., -8.2507e-01,
           -7.5133e-01, -7.9474e-01],
          [-7.8321e-01, -7.8093e-01, -8.4727e-01,  ..., -7.7913e-01,
           -9.3948e-01, -7.9529e-01]],

         [[-8.2015e-02, -8.1217e-02, -5.5365e-02,  ...,  3.5875e-01,
            3.5054e-01,  3.9710e-01],
          [-4.7548e-02, -7.5963e-02, -5.5356e-02,  ...,  3.5545e-01,
            3.8098e-01,  3.7146e-01],
          [-8.5725e-02, -9.1165e-02, -6.4093e-02,  ...,  3.5505e-01,
            3.7086e-01,  3.6016e-01],
          ...,
     